# JOCS

Grobid Data Report

### Cross-Check Trash DOIs vs Final JSONL

In [ ]:
import pandas as pd
import json
import os
from pathlib import Path

# Paths
trash_csv = "jocs_trash_files_only.csv"
jsonl_file = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\data\processed\jocs_final_complete_authors.jsonl")

def fix_filename_to_doi(filename):
    # 1. Extension (.json) hatao
    name = filename.replace('.json', '')
    
    # 2. Year prefix (e.g., 2010_) hatao
    # Hum pehla underscore dhoond kar uske baad ka hissa lenge
    if '_' in name:
        parts = name.split('_', 1)
        doi_part = parts[1] # e.g., '10.1016_j.jocs.2010.04.003'
        
        # 3. Pehle underscore ko forward slash se badlo
        # Result: '10.1016/j.jocs.2010.04.003'
        normalized = doi_part.replace('_', '/', 1)
        return normalized
    return name

def verify_cleanup():
    if not os.path.exists(trash_csv) or not jsonl_file.exists():
        print("❌ Files missing! Check paths.")
        return

    # CSV load karo
    trash_df = pd.read_csv(trash_csv)
    trash_dois = set(trash_df['filename'].apply(fix_filename_to_doi))
    
    print(f"Checking {len(trash_dois)} Trash DOIs against final JSONL...")

    matches = []
    total_jsonl_count = 0

    # JSONL check
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line in f:
            total_jsonl_count += 1
            try:
                data = json.loads(line)
                # normalized_doi check kar rahe hain
                file_doi = data.get('normalized_doi', "").strip()
                
                if file_doi in trash_dois:
                    matches.append(file_doi)
            except:
                continue

    # --- REPORT ---
    print("\n" + "="*60)
    print(f"DATA INTEGRITY REPORT")
    print("-" * 60)
    print(f"Total DOIs in Final JSONL: {total_jsonl_count}")
    print(f"Trash DOIs checked:        {len(trash_dois)}")
    
    if not matches:
        print("\nMUBARAK HO! Logic fix ke baad bhi koi trash match nahi mila.")
        print("Iska matlab 'Situation 8' wali kachra files already filter out ho chuki hain.")
    else:
        print(f"\n🚩 ALERT: {len(matches)} files abhi bhi final data mein baithi hain!")
        for m in matches:
            print(f"  -> Found: {m}")
    print("="*60)

if __name__ == "__main__":
    verify_cleanup()

Checking 53 Trash DOIs against final JSONL...

📊 DATA INTEGRITY REPORT
------------------------------------------------------------
Total DOIs in Final JSONL: 2077
Trash DOIs checked:        53

🚩 ALERT: 40 files abhi bhi final data mein baithi hain!
  -> Found: 10.1016/j.jocs.2024.102379
  -> Found: 10.1016/j.jocs.2023.102183
  -> Found: 10.1016/j.jocs.2023.102102
  -> Found: 10.1016/j.jocs.2022.101745
  -> Found: 10.1016/j.jocs.2020.101227
  -> Found: 10.1016/j.jocs.2020.101258
  -> Found: 10.1016/j.jocs.2021.101395
  -> Found: 10.1016/j.jocs.2020.101222
  -> Found: 10.1016/j.jocs.2020.101187
  -> Found: 10.1016/j.jocs.2020.101168
  -> Found: 10.1016/j.jocs.2019.05.004
  -> Found: 10.1016/j.jocs.2019.04.006
  -> Found: 10.1016/j.jocs.2019.05.006
  -> Found: 10.1016/j.jocs.2019.05.005
  -> Found: 10.1016/j.jocs.2019.03.002
  -> Found: 10.1016/j.jocs.2017.11.014
  -> Found: 10.1016/j.jocs.2018.08.010
  -> Found: 10.1016/j.jocs.2018.04.014
  -> Found: 10.1016/j.jocs.2018.02.005
  -> Fou

In [ ]:
import json
import pandas as pd
import os
import sys
from pathlib import Path

# Path Configuration
INPUT_CSV = "jocs_trash_files_only.csv"
INPUT_JSONL = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\data\processed\jocs_final_complete_authors.jsonl")
OUTPUT_JSONL = INPUT_JSONL.parent / "final_jocs_openalex.jsonl"

def normalize_doi_from_filename(filename):
    """
    Converts filename format to normalized DOI format.
    Example: '2010_10.1016_j.jocs.2010.04.003.json' -> '10.1016/j.jocs.2010.04.003'
    """
    # Remove file extension
    name_no_ext = os.path.splitext(filename)[0]
    
    if '_' in name_no_ext:
        # Split at the first underscore to remove the year prefix
        _, doi_part = name_no_ext.split('_', 1)
        # Replace the first underscore in the remaining string with a forward slash
        normalized_doi = doi_part.replace('_', '/', 1)
        return normalized_doi
    
    return name_no_ext

def generate_final_dataset():
    """
    Filters the input JSONL file by removing records present in the exclusion CSV.
    Writes the resulting clean data to a new JSONL file.
    """
    if not os.path.exists(INPUT_CSV):
        print(f"Error: Required exclusion file '{INPUT_CSV}' not found.")
        sys.exit(1)

    if not INPUT_JSONL.exists():
        print(f"Error: Input dataset '{INPUT_JSONL}' not found.")
        sys.exit(1)

    # Load exclusion list into a set for O(1) lookup performance
    try:
        df_trash = pd.read_csv(INPUT_CSV)
        exclusion_set = set(df_trash['filename'].apply(normalize_doi_from_filename))
        print(f"Exclusion set initialized with {len(exclusion_set)} DOIs.")
    except Exception as e:
        print(f"Error reading exclusion CSV: {e}")
        sys.exit(1)

    records_removed = 0
    records_retained = 0

    # Stream process the JSONL to handle large data volumes
    try:
        with open(INPUT_JSONL, 'r', encoding='utf-8') as f_in, \
             open(OUTPUT_JSONL, 'w', encoding='utf-8') as f_out:
            
            for line_number, line in enumerate(f_in, 1):
                try:
                    record = json.loads(line)
                    doi = record.get('normalized_doi', "").strip()
                    
                    if doi in exclusion_set:
                        records_removed += 1
                        continue
                    
                    # Write clean records back to the new file
                    f_out.write(json.dumps(record, ensure_ascii=False) + '\n')
                    records_retained += 1
                except json.JSONDecodeError:
                    print(f"Warning: Skipping malformed JSON on line {line_number}")
                    continue

        print("\n" + "="*60)
        print("DATASET PURIFICATION COMPLETE")
        print("-" * 60)
        print(f"Output File:      {OUTPUT_JSONL.name}")
        print(f"Records Excluded: {records_removed}")
        print(f"Records Retained: {records_retained}")
        print(f"Full Path:        {OUTPUT_JSONL}")
        print("="*60)

    except Exception as e:
        print(f"An error occurred during file processing: {e}")

if __name__ == "__main__":
    generate_final_dataset()

In [14]:
import os
import json
import csv
from pathlib import Path

# Setup Paths
parsed_path = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\parsed\jocs_grobid\all")
trash_csv = "jocs_trash_files_only.csv"

def print_only_title_files(folder, skip_csv):
    # 1. CSV se skip hone wali files ki list banao
    skip_list = set()
    if os.path.exists(skip_csv):
        with open(skip_csv, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                skip_list.add(row['filename'])
    
    found_files = []

    print(f"Scanning for 'Only Title' files (Skipping files from {skip_csv})...\n")

    for file_path in folder.glob("*.json"):
        # Agar file CSV mein hai, toh skip karo
        if file_path.name in skip_list:
            continue
            
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except:
                continue
            
            # Logic: Title True, Abstract False, Keywords False
            title_text = str(data.get('title') or "").strip()
            has_title = bool(title_text)
            has_kywd = bool(data.get('keywords'))
            has_abst = bool(str(data.get('abstract') or "").strip())

            if has_title and not has_kywd and not has_abst:
                found_files.append({
                    "name": file_path.name,
                    "title": title_text[:80] + "..." if len(title_text) > 80 else title_text
                })

    # --- RESULTS PRINTING ---
    if not found_files:
        print("✅ Koi aisi file nahi mili jo sirf title wali ho (aur trash mein na ho).")
    else:
        print(f"🚀 Found {len(found_files)} files with ONLY TITLE:")
        print("-" * 100)
        print(f"{'Filename':<40} | {'Extracted Title'}")
        print("-" * 100)
        for file in found_files:
            print(f"{file['name']:<40} | {file['title']}")
        print("-" * 100)
        print(f"Total: {len(found_files)} files.")

if __name__ == "__main__":
    print_only_title_files(parsed_path, trash_csv)

Scanning for 'Only Title' files (Skipping files from jocs_trash_files_only.csv)...

✅ Koi aisi file nahi mili jo sirf title wali ho (aur trash mein na ho).


Docling Dataset

In [3]:
import pandas as pd
import json
from pathlib import Path

# --- CONFIGURATION ---
jocs_folder = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\parsed\jocs\all")
output_dir = Path("jocs_analysis")
output_dir.mkdir(exist_ok=True)

print("Scanning JOCS folder...")
all_files = list(jocs_folder.glob("*.json"))

papers = []
for json_file in all_files:
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            # 1. Extraction with Year Fix
            # Filename starts with "2010_", so we split by underscore and take the first part
            filename_year = json_file.name.split('_')[0]
            
            paper_info = {
                'filename': json_file.name,
                'year': int(filename_year) if filename_year.isdigit() else 0,
                'title': data.get('title', ''),
                'abstract': data.get('abstract', ''),
                'keywords': data.get('keywords', ''),
                # 'author_count': len(data.get('authors', [])) # Bonus: keep track of team size
            }
            
            # Combine for NLP/ML analysis
            paper_info['full_text'] = f"{paper_info['title']} {paper_info['abstract']} {paper_info['keywords']}"
            
            papers.append(paper_info)
            
    except Exception as e:
        print(f"Skipped {json_file.name}: {e}")

df = pd.DataFrame(papers)

# --- CLEANING LOGIC ---
# Usable papers: Needs a title and a meaningful abstract
usable_mask = (df['title'].str.len() > 10) & (df['abstract'].str.len() > 30)
df_clean = df[usable_mask].copy()

# --- SAVE ---
df_clean.to_csv(output_dir / "jocs_docling_cleaned.csv", index=False)
print(f"Success! Loaded {len(df)} papers. {len(df_clean)} are high quality.")

Scanning JOCS folder...
Success! Loaded 2094 papers. 1990 are high quality.


In [4]:
fields = ['title', 'keywords', 'abstract']

print("MISSING DATA AUDIT")

for field in fields:
    # Check for both actual NaNs and empty strings/whitespace
    is_missing = df[field].isna() | (df[field].astype(str).str.strip() == "")
    missing_count = is_missing.sum()
    missing_pct = (missing_count / len(df)) * 100
    
    print(f"{field.capitalize():<10}: {missing_count:>4} papers missing ({missing_pct:>5.1f}%)")

# Check for "Completely Empty" rows (missing all three)
all_missing = (
    (df['title'].isna() | (df['title'].astype(str).str.strip() == "")) &
    (df['keywords'].isna() | (df['keywords'].astype(str).str.strip() == "")) &
    (df['abstract'].isna() | (df['abstract'].astype(str).str.strip() == ""))
).sum()


print(f"Total Completely Empty: {all_missing} papers")

MISSING DATA AUDIT
Title     :   67 papers missing (  3.2%)
Keywords  :   90 papers missing (  4.3%)
Abstract  :   63 papers missing (  3.0%)
Total Completely Empty: 25 papers


Filling the missing data from both dataset

In [5]:
import pandas as pd
import os

# 1. Load Data
# Apne paths confirm kar lena
docling_path = r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_docling_cleaned.csv"
grobid_path = r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_grobid_cleaned.csv"

# Load CSVs
print("Loading datasets...")
df_doc = pd.read_csv(docling_path)
df_grob = pd.read_csv(grobid_path)

def check_smushed_abstracts(df, source_name, threshold=0.05):
    print(f"\nScanning {source_name} for Smushed Text...")
    print(f"{'Filename / DOI':<50} | {'Space %':<8} | {'Snippet (First 80 chars)'}")
    print("-" * 120)
    
    bad_count = 0
    total_checked = 0
    
    for index, row in df.iterrows():
        text = str(row.get('abstract', ''))
        
        # Sirf tab check karo agar text maujood hai aur "nan" nahi hai
        if text and text.lower() != 'nan' and len(text) > 50:
            total_checked += 1
            
            # Logic: Count spaces vs Total Length
            space_count = text.count(' ')
            ratio = space_count / len(text)
            
            # Agar spaces 5% se kam hain (Yani 100 letters mein 5 se kam spaces)
            if ratio < threshold:
                bad_count += 1
                # Filename ya DOI jo bhi available ho
                ide = str(row.get('doi', row.get('filename', 'Unknown')))[:45]
                snippet = text[:80].replace('\n', ' ') # Print ke liye newline hata dein
                
                print(f"{ide:<50} | {ratio:.2%}   | {snippet}...")
    
    print("-" * 120)
    print(f"REPORT FOR {source_name}:")
    print(f"Total Abstracts Checked: {total_checked}")
    print(f"Smushed/Corrupted Found: {bad_count} ({(bad_count/total_checked)*100:.2f}%)")

# 2. Run Audit on Docling (Main Culprit)
check_smushed_abstracts(df_doc, "DOCLING")

# 3. Optional: Run on Grobid just to be sure
# check_smushed_abstracts(df_grob, "GROBID")

Loading datasets...

Scanning DOCLING for Smushed Text...
Filename / DOI                                     | Space %  | Snippet (First 80 chars)
------------------------------------------------------------------------------------------------------------------------
2024_10.1016_j.jocs.2024.102466.json               | 3.91%   | Dataset link:https://github.com/AGN000/PIN Addressingdiscontinuitiesinfluidflowp...
2024_10.1016_j.jocs.2024.102469.json               | 4.97%   | Keywords: WestudythetransitiontophasesynchronizationinanensembleofStuart–Landauo...
2025_10.1016_j.jocs.2024.102468.json               | 4.49%   | Keywords: Black-boxauto-tuningmethodshavebeenproventobeefficientfortuningconfigu...
2025_10.1016_j.jocs.2024.102473.json               | 2.73%   | Keywords: Identifying influential nodes is crucial in network science for contro...
2025_10.1016_j.jocs.2024.102498.json               | 2.41%   | Keywords: Theenhancedcapabilitiesofautonomousunderwatervehicles(AUVs)willfacilit.

In [25]:
import pandas as pd
import numpy as np

# 1. Load your two datasets
df_docling = pd.read_csv(r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_docling_cleaned.csv")
df_grobid = pd.read_csv(r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_grobid_cleaned.csv")

# 2. Perform an "Outer Join" on filename to include every paper found by either parser
df_combined = pd.merge(df_grobid, df_docling, on='filename', how='outer', suffixes=('_g', '_d'))

# Helper function to pick the "Better" (longer/non-null) string
def pick_best(val_g, val_d):
    # Convert to string and handle NaNs/None
    s_g = str(val_g).strip() if pd.notna(val_g) else ""
    s_d = str(val_d).strip() if pd.notna(val_d) else ""
    
    # If one is empty, return the other. If both have text, return the longest one.
    if not s_g: return s_d
    if not s_d: return s_g
    return s_g if len(s_g) >= len(s_d) else s_d

# 3. Apply the "Mutual Healing" logic to all key fields
print("Repairing Titles, Abstracts, and Keywords...")
df_combined['final_title'] = df_combined.apply(lambda x: pick_best(x['title_g'], x['title_d']), axis=1)
df_combined['final_abstract'] = df_combined.apply(lambda x: pick_best(x['abstract_g'], x['abstract_d']), axis=1)
df_combined['final_keywords'] = df_combined.apply(lambda x: pick_best(x['keywords_g'], x['keywords_d']), axis=1)

# 4. Repair the Year (Take either, prioritizing Grobid's extraction if valid)
df_combined['final_year'] = df_combined['year_g'].fillna(df_combined['year_d']).replace(0, np.nan)

# 5. Final Cleanup
df_final = df_combined[[
    'filename', 
    'final_title', 
    'final_abstract', 
    'final_keywords', 
    'final_year'
]].copy()

df_final.columns = ['filename', 'title', 'abstract', 'keywords', 'year']

# 6. Final Filter: Remove rows where both title AND abstract are still missing (the 'completely empty' ones)
df_final = df_final.dropna(subset=['title', 'abstract'], how='all')

# Output Results
print(f"\nMERGE COMPLETE")
print(f"Total Papers in Master Dataset: {len(df_final)}")
print(f"Repaired Year Range: {int(df_final['year'].min())} - {int(df_final['year'].max())}")

# Save the final masterpiece
output_path = r"D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_master_dataset.csv"
df_final.to_csv(output_path, index=False)
print(f"Master file saved to: {output_path}")

Repairing Titles, Abstracts, and Keywords...

MERGE COMPLETE
Total Papers in Master Dataset: 2026
Repaired Year Range: 2010 - 2025
Master file saved to: D:\ITMO Big Data & ML School\semester 3\RI3\notebooks\jocs_analysis\jocs_master_dataset.csv


In [27]:
df_audit = df_final 

# Define the fields to check
fields = ['title', 'keywords', 'abstract']

print("FINAL MASTER DATASET AUDIT")
print("-" * 40)

for field in fields:
    # Check for both actual NaNs and empty strings/whitespace
    # We use .fillna('') first to ensure the string strip doesn't fail on NaNs
    is_missing = df_audit[field].fillna('').astype(str).str.strip() == ""
    missing_count = is_missing.sum()
    missing_pct = (missing_count / len(df_audit)) * 100
    
    print(f"{field.capitalize():<10}: {missing_count:>4} papers missing ({missing_pct:>5.1f}%)")

# 1. Check for "Completely Empty" (missing all three metadata fields)
all_missing_mask = (
    (df_audit['title'].fillna('').str.strip() == "") &
    (df_audit['keywords'].fillna('').str.strip() == "") &
    (df_audit['abstract'].fillna('').str.strip() == "")
)
all_missing_count = all_missing_mask.sum()

# 2. Check for "Usable" (Has Title AND Abstract)
# This is your key metric for Big Data / ML School
usable_mask = (
    (df_audit['title'].fillna('').str.strip() != "") & 
    (df_audit['abstract'].fillna('').str.strip().str.len() > 50)
)
usable_count = usable_mask.sum()

print(f"Total Completely Empty : {all_missing_count} papers")
print(f"Total Usable for ML    : {usable_count} papers (Title + Abstract)")

FINAL MASTER DATASET AUDIT
----------------------------------------
Title     :    0 papers missing (  0.0%)
Keywords  :   25 papers missing (  1.2%)
Abstract  :    0 papers missing (  0.0%)
Total Completely Empty : 0 papers
Total Usable for ML    : 2026 papers (Title + Abstract)


In [28]:
# Look for rows where one parser was empty but the final is full
saved_rows = df_combined[df_combined['abstract_g'].isna() & df_combined['final_abstract'].notna()]

print(f"Number of papers saved specifically by Docling: {len(saved_rows)}")
# Display the first 3 examples to see the titles and abstracts
display(saved_rows[['filename', 'final_title', 'final_abstract']].head(3))

Number of papers saved specifically by Docling: 89


,filename,final_title,final_abstract
3,2010_10.1016_j.jocs.2010.03.004.json,Unstable periodic orbits in weak turbulence,We report on a sophisticated numerical study o...
9,2010_10.1016_j.jocs.2010.03.011.json,A dynamic wind farm aggregate model for the si...,Animportantaspect related to wind energy integ...
72,2012_10.1016_j.jocs.2011.06.003.json,Quadpack computation of Feynman loop integrals,The paper addresses a numerical computation of...


# ICCS

GROBID

In [1]:
import pandas as pd
import json
from pathlib import Path

# --- CONFIGURATION ---
# POINTING TO THE ROOT OF THE ICCS GROBID FOLDER
iccs_folder = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\parsed\iccs")
output_dir = Path("iccs_analysis")
output_dir.mkdir(exist_ok=True)

print(f"Scanning ICCS Grobid folder: {iccs_folder}")
# CHANGED: .rglob to find json files inside year subfolders
all_files = list(iccs_folder.rglob("*.json"))

papers = []
for json_file in all_files:
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            # 1. Year Logic
            # Try to get year from folder name first (e.g. .../2001/file.json)
            # If that fails, try filename split
            parent_folder = json_file.parent.name
            if parent_folder.isdigit():
                year = int(parent_folder)
            else:
                # Fallback to filename parsing
                part = json_file.name.split('_')[0]
                year = int(part) if part.isdigit() else 0
            
            paper_info = {
                'filename': json_file.name,
                'year': year,
                'title': data.get('title', ''),
                'abstract': data.get('abstract', ''),
                'keywords': data.get('keywords', ''),
                'source': 'grobid'
            }
            
            # Combine for NLP/ML analysis
            paper_info['full_text'] = f"{paper_info['title']} {paper_info['abstract']} {paper_info['keywords']}"
            
            papers.append(paper_info)
            
    except Exception as e:
        print(f"Skipped {json_file.name}: {e}")

df = pd.DataFrame(papers)

# --- CLEANING LOGIC ---
usable_mask = (df['title'].str.len() > 10) & (df['abstract'].str.len() > 30)
df_clean = df[usable_mask].copy()

# --- SAVE ---
df_clean.to_csv(output_dir / "iccs_grobid_cleaned.csv", index=False)
print(f"\n[GROBID] Success! Loaded {len(df)} papers. {len(df_clean)} are high quality.")

fields = ['title', 'keywords', 'abstract']
print("-" * 30)
print("MISSING DATA AUDIT (GROBID)")
print("-" * 30)

for field in fields:
    is_missing = df[field].isna() | (df[field].astype(str).str.strip() == "")
    missing_count = is_missing.sum()
    missing_pct = (missing_count / len(df)) * 100
    print(f"{field.capitalize():<10}: {missing_count:>4} papers missing ({missing_pct:>5.1f}%)")

all_missing = (
    (df['title'].astype(str).str.strip() == "") &
    (df['keywords'].astype(str).str.strip() == "") &
    (df['abstract'].astype(str).str.strip() == "")
).sum()

print(f"Total Completely Empty: {all_missing} papers")

Scanning ICCS Grobid folder: D:\ITMO Big Data & ML School\semester 3\RI3\parsed\iccs

[GROBID] Success! Loaded 8324 papers. 8118 are high quality.
------------------------------
MISSING DATA AUDIT (GROBID)
------------------------------
Title     :  104 papers missing (  1.2%)
Keywords  : 3075 papers missing ( 36.9%)
Abstract  :  189 papers missing (  2.3%)
Total Completely Empty: 82 papers


DOCLING

In [2]:
import pandas as pd
import json
from pathlib import Path

# --- CONFIGURATION ---
# POINTING TO THE ROOT OF THE ICCS DOCLING FOLDER
# Note: Ensure the path matches your actual folder name (icss_docling vs iccs_docling)
docling_folder = Path(r"D:\ITMO Big Data & ML School\semester 3\RI3\parsed\icss_docling")
output_dir = Path("iccs_analysis")
output_dir.mkdir(exist_ok=True)

print(f"\nScanning ICCS Docling folder: {docling_folder}")
# CHANGED: .rglob to find json files inside year subfolders
all_files = list(docling_folder.rglob("*.json"))

papers = []
for json_file in all_files:
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            # 1. Year Logic (Folder based is safer)
            parent_folder = json_file.parent.name
            if parent_folder.isdigit():
                year = int(parent_folder)
            else:
                part = json_file.name.split('_')[0]
                year = int(part) if part.isdigit() else 0
            
            paper_info = {
                'filename': json_file.name,
                'year': year,
                'title': data.get('title', ''),
                'abstract': data.get('abstract', ''),
                'keywords': data.get('keywords', ''),
                'source': 'docling'
            }
            
            paper_info['full_text'] = f"{paper_info['title']} {paper_info['abstract']} {paper_info['keywords']}"
            
            papers.append(paper_info)
            
    except Exception as e:
        print(f"Skipped {json_file.name}: {e}")

df = pd.DataFrame(papers)

# --- CLEANING LOGIC ---
usable_mask = (df['title'].str.len() > 10) & (df['abstract'].str.len() > 30)
df_clean = df[usable_mask].copy()

# --- SAVE ---
df_clean.to_csv(output_dir / "iccs_docling_cleaned.csv", index=False)
print(f"\n[DOCLING] Success! Loaded {len(df)} papers. {len(df_clean)} are high quality.")

fields = ['title', 'keywords', 'abstract']
print("-" * 30)
print("MISSING DATA AUDIT (DOCLING)")
print("-" * 30)

for field in fields:
    is_missing = df[field].isna() | (df[field].astype(str).str.strip() == "")
    missing_count = is_missing.sum()
    missing_pct = (missing_count / len(df)) * 100
    print(f"{field.capitalize():<10}: {missing_count:>4} papers missing ({missing_pct:>5.1f}%)")

all_missing = (
    (df['title'].astype(str).str.strip() == "") &
    (df['keywords'].astype(str).str.strip() == "") &
    (df['abstract'].astype(str).str.strip() == "")
).sum()

print(f"Total Completely Empty: {all_missing} papers")


Scanning ICCS Docling folder: D:\ITMO Big Data & ML School\semester 3\RI3\parsed\icss_docling

[DOCLING] Success! Loaded 6135 papers. 5904 are high quality.
------------------------------
MISSING DATA AUDIT (DOCLING)
------------------------------
Title     :    2 papers missing (  0.0%)
Keywords  : 3048 papers missing ( 49.7%)
Abstract  :  199 papers missing (  3.2%)
Total Completely Empty: 0 papers


Merging

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- CONFIGURATION ---
# We assume the CSVs were generated in the 'iccs_analysis' folder by the previous scripts
GROBID_CSV_PATH = Path("iccs_analysis/iccs_grobid_cleaned.csv")
DOCLING_CSV_PATH = Path("iccs_analysis/iccs_docling_cleaned.csv")
OUTPUT_MASTER_PATH = Path("iccs_analysis/iccs_master_dataset.csv")

# 1. Load your two datasets
print("Loading datasets...")
if not GROBID_CSV_PATH.exists() or not DOCLING_CSV_PATH.exists():
    print(f"ERROR: One or both input files not found in 'iccs_analysis/'.")
    print(f"Check: {GROBID_CSV_PATH} and {DOCLING_CSV_PATH}")
    exit()

df_grobid = pd.read_csv(GROBID_CSV_PATH)
df_docling = pd.read_csv(DOCLING_CSV_PATH)

# Ensure filenames are strings for consistent merging
df_grobid['filename'] = df_grobid['filename'].astype(str)
df_docling['filename'] = df_docling['filename'].astype(str)

print(f"Grobid papers: {len(df_grobid)}")
print(f"Docling papers: {len(df_docling)}")

# 2. Perform an "Outer Join" on filename to include every paper found by either parser
df_combined = pd.merge(df_grobid, df_docling, on='filename', how='outer', suffixes=('_g', '_d'))

# Helper function to pick the "Better" (longer/non-null) string
def pick_best(val_g, val_d):
    # Convert to string and handle NaNs/None
    s_g = str(val_g).strip() if pd.notna(val_g) and str(val_g).lower() != 'nan' else ""
    s_d = str(val_d).strip() if pd.notna(val_d) and str(val_d).lower() != 'nan' else ""
    
    # If one is empty, return the other. If both have text, return the longest one.
    if not s_g: return s_d
    if not s_d: return s_g
    return s_g if len(s_g) >= len(s_d) else s_d

# 3. Apply the "Mutual Healing" logic to all key fields
print("Repairing Titles, Abstracts, and Keywords...")
df_combined['final_title'] = df_combined.apply(lambda x: pick_best(x['title_g'], x['title_d']), axis=1)
df_combined['final_abstract'] = df_combined.apply(lambda x: pick_best(x['abstract_g'], x['abstract_d']), axis=1)
df_combined['final_keywords'] = df_combined.apply(lambda x: pick_best(x['keywords_g'], x['keywords_d']), axis=1)

# 4. Repair the Year (Take either, prioritizing Grobid's extraction if valid)
# This prioritization works because Grobid covered all years consistently
df_combined['final_year'] = df_combined['year_g'].fillna(df_combined['year_d']).replace(0, np.nan)

# 5. Final Cleanup
df_final = df_combined[[
    'filename', 
    'final_title', 
    'final_abstract', 
    'final_keywords', 
    'final_year'
]].copy()

df_final.columns = ['filename', 'title', 'abstract', 'keywords', 'year']

# 6. Final Filter: Remove rows where both title AND abstract are still missing (the 'completely empty' ones)
df_final = df_final.dropna(subset=['title', 'abstract'], how='all')

# Output Results
print(f"\nMERGE COMPLETE")
print(f"Total Papers in Master Dataset: {len(df_final)}")
if df_final['year'].notna().any():
    print(f"Repaired Year Range: {int(df_final['year'].min())} - {int(df_final['year'].max())}")

# Save the final masterpiece
df_final.to_csv(OUTPUT_MASTER_PATH, index=False)
print(f"Master file saved to: {OUTPUT_MASTER_PATH}")

# --- AUDIT SECTION ---
df_audit = df_final 

# Define the fields to check
fields = ['title', 'keywords', 'abstract']

print("\nFINAL MASTER DATASET AUDIT")
print("-" * 40)

for field in fields:
    # Check for both actual NaNs and empty strings/whitespace
    # We use .fillna('') first to ensure the string strip doesn't fail on NaNs
    is_missing = df_audit[field].fillna('').astype(str).str.strip() == ""
    missing_count = is_missing.sum()
    missing_pct = (missing_count / len(df_audit)) * 100
    
    print(f"{field.capitalize():<10}: {missing_count:>4} papers missing ({missing_pct:>5.1f}%)")

# 1. Check for "Completely Empty" (missing all three metadata fields)
all_missing_mask = (
    (df_audit['title'].fillna('').str.strip() == "") &
    (df_audit['keywords'].fillna('').str.strip() == "") &
    (df_audit['abstract'].fillna('').str.strip() == "")
)
all_missing_count = all_missing_mask.sum()

# 2. Check for "Usable" (Has Title AND Abstract)
# This is your key metric for Big Data / ML School
usable_mask = (
    (df_audit['title'].fillna('').str.strip() != "") & 
    (df_audit['abstract'].fillna('').str.strip().str.len() > 50)
)
usable_count = usable_mask.sum()

print("-" * 40)
print(f"Total Completely Empty : {all_missing_count} papers")
print(f"Total Usable for ML    : {usable_count} papers (Title + Abstract)")

Loading datasets...
Grobid papers: 8118
Docling papers: 5904
Repairing Titles, Abstracts, and Keywords...

MERGE COMPLETE
Total Papers in Master Dataset: 8181
Repaired Year Range: 2001 - 2025
Master file saved to: iccs_analysis\iccs_master_dataset.csv

FINAL MASTER DATASET AUDIT
----------------------------------------
Title     :    0 papers missing (  0.0%)
Keywords  : 2913 papers missing ( 35.6%)
Abstract  :    0 papers missing (  0.0%)
----------------------------------------
Total Completely Empty : 0 papers
Total Usable for ML    : 8181 papers (Title + Abstract)


Audit

In [ ]:
import pandas as pd
from pathlib import Path

# Load your files
grobid_path = Path("iccs_analysis/iccs_grobid_cleaned.csv")
docling_path = Path("iccs_analysis/iccs_docling_cleaned.csv")
master_path = Path("iccs_analysis/iccs_master_dataset.csv")

df_g = pd.read_csv(grobid_path).set_index("filename")
df_d = pd.read_csv(docling_path).set_index("filename")
df_m = pd.read_csv(master_path).set_index("filename")

print(f"Loaded Master Dataset: {len(df_m)} records\n")

# --- TEST 1: Did we actually fix broken Titles? ---
# Find papers where Grobid title was missing/short, but Master has it.
print("--- TEST 1: TITLE REPAIRS ---")
bad_grobid_title = df_g[df_g['title'].str.len() < 10].index
repaired_titles = df_m.loc[df_m.index.intersection(bad_grobid_title)]

if len(repaired_titles) > 0:
    sample = repaired_titles.iloc[0]
    fname = sample.name
    print(f"File: {fname}")
    print(f"BEFORE (Grobid): {df_g.loc[fname, 'title']}")
    print(f"AFTER  (Master): {sample['title']}")
    print(f"Repaired {len(repaired_titles)} titles using Docling.")
else:
    print("No title repairs found (Grobid titles were already good or dropped).")

print("-" * 30)

# --- TEST 2: Did we actually fix broken Abstracts? ---
# Find papers where Grobid abstract was missing/short, but Master has a long one.
print("--- TEST 2: ABSTRACT REPAIRS ---")
bad_grobid_abs = df_g[df_g['abstract'].str.len() < 50].index
repaired_abs = df_m.loc[df_m.index.intersection(bad_grobid_abs)]

# Filter for ones that are actually good now
repaired_abs = repaired_abs[repaired_abs['abstract'].str.len() > 100]

if len(repaired_abs) > 0:
    sample = repaired_abs.iloc[0]
    fname = sample.name
    print(f"File: {fname}")
    print(f"BEFORE (Grobid): {df_g.loc[fname, 'abstract']}")
    print(f"AFTER  (Master): {sample['abstract'][:200]}...") # truncate for display
    print(f"Repaired {len(repaired_abs)} abstracts using Docling.")
else:
    print("No abstract repairs found.")

print("-" * 30)

# --- TEST 3: Did we get the longer text? ---
# Check 5 random common files to see if text length improved
print("--- TEST 3: LENGTH COMPARISON (Docling vs Grobid) ---")
common_files = df_g.index.intersection(df_d.index)
if len(common_files) > 0:
    sample_files = common_files[:5]
    for fname in sample_files:
        len_g = len(str(df_g.loc[fname, 'abstract']))
        len_m = len(str(df_m.loc[fname, 'abstract']))
        
        status = "SAME"
        if len_m > len_g: status = "IMPROVED"
        
        print(f"File: {fname[:30]}... | Grobid: {len_g} chars | Master: {len_m} chars -> {status}")

Loaded Master Dataset: 8182 records

--- TEST 1: TITLE REPAIRS ---
No title repairs found (Grobid titles were already good or dropped).
------------------------------
--- TEST 2: ABSTRACT REPAIRS ---
File: 2022_10.1007_978-3-031-08757-8_50.json
BEFORE (Grobid): Graphics-Visualization-Computing Lab (GVCL),
AFTER  (Master): Automated interpretation of digital images of charts in documents and the internet helps to improve the accessibility of visual representation of data. One of the approaches for automation involves ex...
Repaired 1 abstracts using Docling.
------------------------------
--- TEST 3: LENGTH COMPARISON (Docling vs Grobid) ---
File: 2001_10.1007_3-540-45545-0_1.j... | Grobid: 1173 chars | Master: 1173 chars -> SAME
File: 2001_10.1007_3-540-45545-0_10.... | Grobid: 459 chars | Master: 459 chars -> SAME
File: 2001_10.1007_3-540-45545-0_101... | Grobid: 498 chars | Master: 498 chars -> SAME
File: 2001_10.1007_3-540-45545-0_102... | Grobid: 547 chars | Master: 547 chars -> SA